In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import rasterio
from rasterio.transform import from_origin

In [ ]:
# -----------------------------
# Parâmetros (espelham o .m)
# -----------------------------
desvio_g = 1.0           # mGal (ruído)
resolucao_graus = 0.05   # p/ borda no GeoTIFF (igual ao .m)
d_lat = 0.5              # tamanho do bloco (°)
d_long = 0.5             # tamanho do bloco (°)
offset = 0.25            # margem extra (°)

# Coeficientes polinomiais Xa (grau 9 como no seu exemplo .m)
Xa = np.array([
    226.783646281416,
    -155.004524024261,
    -3343.83283285998,
    9976.21757061898,
    -12499.9037185957,
    8510.70347626656,
    -3389.2325428571,
    789.255025319892,
    -99.6337374628924,
    5.2686882195962,
], dtype=float)

# Arquivos de entrada/saída
p_remove = Path("../REMOVER/remove_pontos.csv")    # lat, lon, dg_res_calc_mgal
p_grade  = Path("../GERACAO_GRADE_ANOMALIAS_RESIDUAIS/Grade_interpolar.txt")
p_tif    = Path("data/processed/Grade_AG_res_Poly9.tif")

In [ ]:

# -----------------------------
# Leitura dos dados
# -----------------------------
rem = pd.read_csv(p_remove)
Lat_all = rem["lat"].to_numpy()
Lon_all = rem["lon"].to_numpy()
G_all   = rem["dg_res_calc_mgal"].to_numpy()
N_all   = np.full_like(G_all, desvio_g, dtype=float)   # Desvios_g no .m

grade = np.loadtxt(p_grade)  # 2 colunas: lat lon
Lat_i_all = grade[:, 0]
Lon_i_all = grade[:, 1]

# -----------------------------
# Funções auxiliares
# -----------------------------
def poly_cov(dist_deg, Xa, deg=None):
    """
    C(dist) = a0 + a1*s + ... + a_n*s^n, com s em GRAUS (fiel ao .m).
    deg=None -> usa len(Xa)-1; senão, trunca.
    """
    if deg is None:
        deg = len(Xa) - 1
    # Vandermonde com potências 0..deg
    powers = np.vstack([dist_deg**k for k in range(deg+1)]).T
    return powers @ Xa[:deg+1]

def lsc_poly(lon_i, lat_i, lon_obs, lat_obs, Xa, N_vec, G_vec, deg=None, nugget=0.0):
    """
    Estima valores em (lon_i, lat_i) a partir dos observados por LSC usando covariância polinomial.
    Distâncias em graus (mesma unidade do .m).
    """
    # Matriz de distâncias obs-obs (graus)
    dx_oo = lon_obs[:, None] - lon_obs[None, :]
    dy_oo = lat_obs[:, None] - lat_obs[None, :]
    s_oo = np.sqrt(dx_oo**2 + dy_oo**2)

    Czz = poly_cov(s_oo, Xa, deg=deg)
    # Regularização leve (se necessário)
    if nugget > 0:
        Czz = Czz + nugget*np.eye(Czz.shape[0])

    # Soma diag(N) igual ao script
    Czz = Czz + np.diag(N_vec)

    # Distâncias grade-obs (graus)
    dx_io = lon_i[:, None] - lon_obs[None, :]
    dy_io = lat_i[:, None] - lat_obs[None, :]
    s_io = np.sqrt(dx_io**2 + dy_io**2)

    Csz = poly_cov(s_io, Xa, deg=deg)

    # Resolve: (Czz)^{-1} G  via solve (mais estável)
    sol = np.linalg.solve(Czz, G_vec)
    G_hat = Csz @ sol
    return G_hat

# -----------------------------
# Preparar blocos de interpolação
# -----------------------------
lat_min_i, lat_max_i = Lat_i_all.min(), Lat_i_all.max()
lon_min_i, lon_max_i = Lon_i_all.min(), Lon_i_all.max()

n_bloc_lat  = int(np.ceil((lat_max_i - lat_min_i) / d_lat))
n_bloc_long = int(np.ceil((lon_max_i - lon_min_i) / d_long))

# Vamos acumular resultados aqui
out_lats, out_lons, out_vals = [], [], []

# -----------------------------
# Loop por blocos (fiel ao .m)
# -----------------------------
for bi in range(n_bloc_lat):
    for bj in range(n_bloc_long):
        # limites do bloco (sem offset)
        min_lat = lat_min_i + bi*d_lat
        max_lat = lat_min_i + (bi+1)*d_lat
        min_lon = lon_min_i + bj*d_long
        max_lon = lon_min_i + (bj+1)*d_long

        # Seleciona observações dentro do bloco + offset (com borda)
        lat_mask = (Lat_all >= (min_lat - offset)) & (Lat_all <= (max_lat + (offset if bi == n_bloc_lat-1 else offset)))
        lon_mask = (Lon_all >= (min_lon - offset)) & (Lon_all <= (max_lon + (offset if bj == n_bloc_long-1 else offset)))
        obs_mask = lat_mask & lon_mask

        lat_obs = Lat_all[obs_mask]
        lon_obs = Lon_all[obs_mask]
        G_obs   = G_all[obs_mask]
        N_obs   = N_all[obs_mask]

        # Seleciona pontos da GRADE estritamente dentro do bloco (sem offset)
        grid_mask = (Lat_i_all >= min_lat) & (Lat_i_all <= (max_lat if bi == n_bloc_lat-1 else max_lat)) & \
                    (Lon_i_all >= min_lon) & (Lon_i_all <= (max_lon if bj == n_bloc_long-1 else max_lon))

        lat_i = Lat_i_all[grid_mask]
        lon_i = Lon_i_all[grid_mask]

        # Se não há pontos de grade nesse bloco, pula
        if lat_i.size == 0:
            continue
        # Se há poucas observações, use vizinhos do bloco inteiro (fallback)
        if lat_obs.size < 4:
            # pega vizinhos num raio maior (ex.: 2*offset) — simples fallback
            extra_mask = (Lat_all >= (min_lat - 2*offset)) & (Lat_all <= (max_lat + 2*offset)) & \
                         (Lon_all >= (min_lon - 2*offset)) & (Lon_all <= (max_lon + 2*offset))
            lat_obs = Lat_all[extra_mask]
            lon_obs = Lon_all[extra_mask]
            G_obs   = G_all[extra_mask]
            N_obs   = N_all[extra_mask]

        # LSC com polinômio inteiro (grau = len(Xa)-1)
        # Obs: no .m eles comentam versões truncadas; aqui usamos full.
        G_hat = lsc_poly(lon_i, lat_i, lon_obs, lat_obs, Xa, N_obs, G_obs, deg=len(Xa)-1, nugget=0.0)

        out_lats.append(lat_i)
        out_lons.append(lon_i)
        out_vals.append(G_hat)

# Concatena resultados
if len(out_vals) == 0:
    raise RuntimeError("Nenhum bloco produziu resultados. Verifique máscaras, arquivos e limites.")
LATS = np.concatenate(out_lats)
LONS = np.concatenate(out_lons)
VALS = np.concatenate(out_vals)

# Ordena por (lat, lon) como no MATLAB antes de reshape
order = np.lexsort((LONS, LATS))
LATS, LONS, VALS = LATS[order], LONS[order], VALS[order]

# Reconstrói a grade
lat_unique = np.unique(Lat_i_all)
lon_unique = np.unique(Lon_i_all)
nlin = lat_unique.size
ncol = lon_unique.size

# Mapeia (LATS,LONS) -> matriz [nlin x ncol]
# (assumindo que Grade_interpolar.txt está em ordem de variação de long mais rápida)
# Vamos garantir índice por grade estruturada:
lat_to_idx = {v:i for i,v in enumerate(lat_unique)}
lon_to_idx = {v:i for i,v in enumerate(lon_unique)}
IMG = np.full((nlin, ncol), np.nan, dtype=float)
for la, lo, va in zip(LATS, LONS, VALS):
    i = lat_to_idx[la]
    j = lon_to_idx[lo]
    IMG[i, j] = va

# -----------------------------
# Escrever GeoTIFF
# Limites com meia-célula (como no .m)
# -----------------------------
lat_S = lat_unique.min() - (resolucao_graus/2)
lat_N = lat_unique.max() + (resolucao_graus/2)
lon_W = lon_unique.min() - (resolucao_graus/2)
lon_E = lon_unique.max() + (resolucao_graus/2)

# Em rasterio, precisamos do transform e da orientação "origem no canto superior-esquerdo"
# Nosso grid está indexado [lat crescente, lon crescente]; para o transform,
# definimos origem em (lon_W, lat_N) e pixel size (dx=+res, dy=+res) mas com linha 0 = topo (lat_N -> decresce).
# A solução simples: escrever invertendo o eixo lat (de cima para baixo).
transform = from_origin(lon_W, lat_N, resolucao_graus, resolucao_graus)
IMG_to_write = np.flipud(IMG)  # inverte no eixo vertical

with rasterio.open(
    p_tif, "w",
    driver="GTiff",
    height=IMG_to_write.shape[0],
    width=IMG_to_write.shape[1],
    count=1,
    dtype="float32",
    crs="EPSG:4326",
    transform=transform,
) as dst:
    dst.write(IMG_to_write.astype("float32"), 1)

print(f"OK! GeoTIFF salvo em: {p_tif.resolve()}")